# A1. US Presidential Speeches 

In [ ]:
# !pip3 install nltk
# !pip3 install contractions

In [ ]:
# import needed libraries
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
import contractions
import string

### Create corpus

In [ ]:
from nltk.corpus import inaugural

speech = inaugural.raw('1789-Washington.txt')

### Add year of speech

### Add party affiliation (Democrat/Republican/Other)

### Preprocessing

In [ ]:
# lowercase
speech = speech.lower()
speech[:200]

In [ ]:
# expand contractions
contractions.add('fellow-citizens', 'fellow citizens')

expanded_words = [contractions.fix(word) for word in speech.split()]
speech = ' '.join(expanded_words)
speech[:200]

In [ ]:
# remove punctuations: using translate
speech = speech.translate(str.maketrans('', '', string.punctuation))
speech[:200]

In [ ]:
# tokenize
tokens = speech.split()

len(tokens), tokens[:5]

In [ ]:
tokens = [token for token in tokens if len(token) > 2]
len(tokens), tokens[:5]

In [ ]:
# remove stopwords
stop_words = set(stopwords.words('english'))

tokens_filtered = [token for token in tokens if not token in stop_words]

len(tokens_filtered), tokens_filtered[:5]

In [ ]:
# remove digits and words combined with digits 


In [ ]:
tokens_tagged = pos_tag(tokens_filtered)

In [ ]:
# Lemmatize with POS Tag

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


# Init Lemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatize filtered_tokens with the appropriate POS tags
sentence = "The striped bats are hanging on their feet for best"
print([lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens_filtered])

In [ ]:
# remove extra spaces
speech

##### Which presidents have the most/least vocabulary

##### Which president has the least vocabulary

##### On average, do Democratic, Republican, or Other presidents have higher vocabulary?

##### Create a barplot of presidential vocabulary from the earliest president (Washington) to the latest (Trump) in chronological order. Color code this barplot as blue for Democrat, red for Republican, and gray for Others. (1 point)

##### What are the five most frequently used words (exclusive of stop words) used by each president? What are the five most frequently words used collectively by all Democratic presidents versus Republican presidents? (2 point)

##### What are the key themes (e.g., freedom, liberty, country, etc.) used by each president in their inaugural speech? (3 points)

##### Compute a sentiment (positive/negative) for each presidential speech, and draw a barplot of the sentiment of all presidential speeches in chronological order. Again, color code the speeches as blue for Democrat, red for Republican, and gray for Other. Which of these groups have higher mean sentiment score? Who are the top three presidents with the highest positive sentiment in each group? (2 points)